# Bitwise XOR operation

In [ ]:
# Constants
N = 4
A_VAL = 0b0011
B_VAL = 0b0101
SHOTS = 1000

## Qiskit Library usage

Only capable of computing bitwise XOR where the control qubits are in the computational basis

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import BitwiseXorGate

xor = BitwiseXorGate(num_qubits=N, amount=B_VAL)

# Create A = |0011>
reg_a = QuantumRegister(N, "a")
A = QuantumCircuit(reg_a)
A.initialize(A_VAL)

# Create a circuit to hold everything, including a classical register for the result
reg_result = ClassicalRegister(N, "res")
circuit = QuantumCircuit(reg_a, reg_result)

circuit = (
    circuit.compose(A, qubits=reg_a)
    .compose(xor)
)
circuit.measure(reg_a, reg_result)
circuit.draw("mpl")

In [ ]:
from qiskit.primitives import StatevectorSampler
 
result = StatevectorSampler().run([circuit], shots=SHOTS).result()
print(f"Count data:\n {result[0].data.res.get_counts()}")

## Custom Implementation

### Running on qubits in their eigenstates

In [ ]:
def bitwiseXOR(qc, reg_a, reg_b):
    assert(reg_a.size == reg_b.size)
    for i in range(reg_a.size):
        qc.cx(reg_a[i], reg_b[i])

# Create A = |0011>
reg_a = QuantumRegister(N, "a")
A = QuantumCircuit(reg_a)
A.initialize(A_VAL)

# Create B = |0101>
reg_b = QuantumRegister(N, "b")
B = QuantumCircuit(reg_b)
B.initialize(B_VAL)

# Create full quantum circuit
qregs = [
    QuantumRegister(N, "a"),
    QuantumRegister(N, "b"),
]
reg_result = ClassicalRegister(N, "res")
circuit = QuantumCircuit(*qregs, reg_result)

circuit = (
    circuit.compose(A, qubits=reg_a)
    .compose(B, qubits=reg_b)
)
bitwiseXOR(circuit, reg_a, reg_b)
circuit.measure(reg_b, reg_result)
circuit.draw("mpl")

In [ ]:
result = StatevectorSampler().run([circuit], shots=SHOTS).result()
print(f"Count data:\n {result[0].data.res.get_counts()}")

## Running on qubits in superposition

In [ ]:
import math

coeff = 1/math.sqrt(2)**N
print(coeff)

# |+>⊗|+>⊗|+>⊗|+>
state_plus2 = [coeff] * (2**N)

reg_a = QuantumRegister(N, "a")
A = QuantumCircuit(reg_a)
A.initialize(state_plus2, reg_a)

reg_b = QuantumRegister(N, "b")
B = QuantumCircuit(reg_b)
B.initialize(state_plus2, reg_b)

# Create full quantum circuit
qregs = [
    QuantumRegister(N, "a"),
    QuantumRegister(N, "b"),
]
reg_result = ClassicalRegister(N, "res")
circuit = QuantumCircuit(*qregs, reg_result)

circuit = (
    circuit.compose(A, qubits=reg_a)
    .compose(B, qubits=reg_b)
)
bitwiseXOR(circuit, reg_a, reg_b)
circuit.measure(reg_b, reg_result)
circuit.draw("mpl")

In [ ]:
result = StatevectorSampler().run([circuit], shots=SHOTS).result()
counts = result[0].data.res.get_counts()
print(f"Count data:\n {counts}")
print(f"Expected count of each: {SHOTS * coeff**2}")

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)